In [2]:
%load_ext autoreload

%autoreload 2

import sys
sys.path.insert(1, "..")

import pandas as pd
import numpy as np

%aimport recordlinkage

In [4]:
dfA = pd.read_csv('../sampledata/dataset2.csv', index_col='rec_id', na_values=' ')
dfB = pd.read_csv('../sampledata/dataset1.csv', index_col='rec_id', na_values=' ')

print "The number of record in dataset A: %s" % len(dfA)
print "The number of record in dataset B: %s" % len(dfB)

dfB.head()

The number of record in dataset A: 5000
The number of record in dataset B: 1000


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [ ]:
std_dfA = recordlinkage.StandardDataFrame(dfA)

Create an object Pairs to create candidate record pairs. The class 'Pairs' can take 1 or 2 arguments, both dataframes. If one dataframe is given, the record pairs are for deduplication. When two dataframes are given, the data is linked between two files. 

In [ ]:
pairing = recordlinkage.Pairs(dfA, dfB, suffixes=('_dfA', '_dfB'))

The simplest method of comparing record pairs is to compare all possible records pairs. The method 'full' is used for this. The function takes no arguments. 

In [ ]:
pairs_full = pairing.full()
print "The reduction ratio is %s" % pairing.reduction_ratio()
pairs_full.head(10)

Make pairs based on a blocking key. This blocking key can be a list of blocking keys. In this case 'Surname' was used. 

In [ ]:
pairs_block = pairing.block(' surname')
print "The reduction ratio is %s" % pairing.reduction_ratio()

pairs_block.head(10)

In [ ]:
pairs_sorted = pairing.sortedneighbourhood(' surname', window=3)
print "The reduction ratio is %s" % pairing.reduction_ratio()

pairs_sorted.head(10)

In [ ]:
pairs_sorted_with_block = pairing.sortedneighbourhood(' given_name', window=3, blocking_on=[' surname'])
pairs_sorted_with_block.head(10)


In [ ]:
for pairs in pairing.iterindex(recordlinkage.indexing._fullindex, 1000,1000):
    
    if not pairs.empty:
        print 'block'

This is the same as the following code.

In [ ]:
for pairs in pairing.iterfull(1000,1000):
    
    pass

Also blocking and sorted neighbourhood indexing can be used with iterations. The number of records pairs is not always equal for each iteration. 

In [ ]:
for pairs in pairing.iterblock(1000,1000, ' surname'):

    pass

In [ ]:
comp = recordlinkage.Compare(pairs_block)

# print pairs_block.columns
comp.compare(recordlinkage.comparing.exact, 
             pairs_block[[' given_name_dfA',' address_2_dfA']], 
             pairs_block[[' given_name_dfB', ' address_2_dfB']], 
             missing_value=9, 
             output='any',
             name='given_name')
comp.compare(recordlinkage.comparing.exact, 
             pairs_block[' given_name_dfA'], 
             pairs_block[' address_2_dfB'], 
             missing_value=9, 
             output='any',
             name='sur_name')
# comp.compare(comparing.exact, pairs_block[' address_2_dfA'], pairs_block[' address_2_dfB'], missing_value=np.nan , name='address')

# pairs_block.loc[pairs_block.ix[[0,1,2,3]].index,' given_name_dfA'] = 3
# print sum(pairs_block[' given_name_dfB'].isnull())

comp.comparison_vectors

In [ ]:
fs = recordlinkage.FellegiSunterClassifier()

fs.estimate(
    comparison_vectors=comp.comparison_vectors, 
    start_m={'given_name':{0:0.1, 1:0.7, 9:0.2},'sur_name':{0:0.1, 9:0.9}},
    start_u={'given_name':{0:0.7, 1:0.1, 9: 0.2},'sur_name':{0:0.9, 9:0.1}},
    start_p=0.1,
    max_iter=10
)

print fs.est.number_of_pairs
print fs.est.p
fs.est.summary()

len(fs.auto_classify(comp.comparison_vectors))





In [ ]:
x = pd.Series([1,2,3,4])
y = pd.Series([1,2,3,4])

x.name= 'test'
y.name = 'red'
print x 
print y

pd.concat([x,y], axis=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import networkx as nx

B = nx.Graph()
B.add_nodes_from([1,2], bipartite=0) # Add the node attribute "bipartite"
B.add_nodes_from([3], bipartite=1)
B.add_edges_from([(1,1), (1,2)])

nx.draw(B)

In [71]:
reload(recordlinkage)
import itertools

test_data = recordlinkage.StandardSeries(dfA[' given_name'].copy())

In [72]:
test_data.group_similar_values()

rec_id
rec-2778-org             sarah
rec-712-dup-0            jacob
rec-1321-org           brinley
rec-3004-org           aleisha
rec-1384-org             ethan
rec-3981-org            alicia
rec-916-org           benjamin
rec-1684-org          petreece
rec-63-dup-0            olivia
rec-3808-org               NaN
rec-112-org             joshua
rec-3297-org           rachael
rec-1315-org            joseph
rec-1050-org             sarah
rec-2116-org           sidonie
rec-3232-org            andrew
rec-1900-dup-1           kiara
rec-2460-dup-2        nicholas
rec-3123-org          isabella
rec-2166-org         alexandra
rec-1155-org           matthew
rec-1485-org           michael
rec-2852-org           georgia
rec-707-org                tia
rec-3499-org            brooke
rec-2153-org              liam
rec-1665-dup-2          jamesr
rec-35-dup-1             darcy
rec-78-org                kody
rec-3116-org             james
                      ...     
rec-3650-org            samuel
r

In [59]:
%timeit list(itertools.combinations(test_data[test_data.notnull()].astype(unicode).unique(), 2))

The slowest run took 4.19 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 255 µs per loop
